# 개인별 추천 목록

* 점수 기반 인기 영화 20개
* 장르별 인기 영화 5개씩
* 연도별 인기 영화 5개씩
* 많이 시청한 영화 20개
* 해당 유저의 연령대별 인기 영화 20개
* 최근 시청한 5개 영화의 연관 영화 25개

In [63]:
from google.colab import drive
drive.mount('/content/drive')

# 불러오기
import pandas as pd
import pickle

output_path = '/content/drive/MyDrive/2307_추천시스템_101/user_rec_dict_recent.pickle'

with open(output_path, 'rb') as f:
  user_result_dict = pickle.load(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
movie_df = pd.read_csv('/content/drive/MyDrive/2307_추천시스템_101/movie_df.csv', index_col=0)
user_df = pd.read_csv('/content/drive/MyDrive/2307_추천시스템_101/user_df.csv', index_col=0)
rating_url = 'https://raw.githubusercontent.com/yoonkt200/python-data-analysis/master/data/ml-1m/ratings.dat'
rating_df = pd.read_csv(rating_url, names=['user_id', 'movie_id', 'rating', 'time'], delimiter='::', engine ='python')
rating_df.head()

,user_id,movie_id,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## 시간대 기반 테스트 데이터 셋 생성

In [65]:
split_bound = rating_df['time'].quantile(q=0.8)
test_df = rating_df[rating_df['time'] >= split_bound]

In [66]:
user_test_watch_list = test_df.groupby('user_id')['movie_id'].apply(set)

In [67]:
user_test_watch_list

user_id
1       {1, 2692, 260, 1028, 1287, 1029, 1545, 1035, 5...
2       {1537, 515, 1544, 2571, 1552, 2067, 21, 3095, ...
3       {260, 648, 1291, 653, 1431, 1304, 1049, 3868, ...
4       {2947, 260, 2692, 2951, 3468, 1036, 1954, 1196...
5       {2560, 515, 6, 3079, 3081, 2058, 3083, 2571, 1...
                              ...                        
6001    {132, 1036, 1552, 1683, 1687, 3481, 1438, 3751...
6002    {2946, 2819, 2947, 2948, 902, 1927, 2949, 1161...
6016    {2944, 900, 3336, 2062, 2704, 2065, 914, 2962,...
6028                                               {3000}
6040    {1921, 1673, 2571, 2575, 2068, 1947, 161, 3362...
Name: movie_id, Length: 1783, dtype: object

## 평가 계획 수립

- 예상 선호 리스트와 실제 시청 리스트로 MAP@K, Recall@K 계산
  - MAP : Mean Average Precision. 각 유저별 precision을 계산한 뒤 모든 추천 대상 유저로 확장해 평균적인 지표로 계산.
  - Recall : 각 유저별 Recall을 계산한 뒤 모든 추천 대상 유저로 확장해 평균적인 지표로 계산.
  
[가설]
유저 A가 실제로 시청한 영화들을 a, 모델이 유저 A가 시청할 것이라고 예측한 영화들을 b라고 가정한다. 모델이 10개의 영화를 예측했다면 Top 10 Precision, recall을 계산한다.


**단, 유저가 단순히 시청만 했다고 영화에 만족했다고 볼 수 없음. 추천 기준을 하나 더 적용**
= 4점 이상인 경우 만족했다고 가정

In [68]:
user_test_like_list = test_df[test_df['rating']>=4].groupby('user_id')['movie_id'].apply(set)

In [69]:
user_test_like_list

user_id
1       {1, 2692, 260, 1028, 1287, 1029, 1545, 1035, 5...
2       {1537, 515, 3334, 648, 1544, 265, 2571, 3468, ...
3       {260, 1291, 653, 1304, 1049, 2081, 2470, 552, ...
4       {480, 2366, 1954, 2947, 260, 2692, 2951, 1097,...
5       {2560, 515, 3083, 2571, 2580, 1046, 29, 32, 34...
                              ...                        
6001    {481, 965, 3751, 2600, 457, 2346, 3947, 3147, ...
6002    {2946, 2819, 2947, 2948, 2949, 1927, 1419, 909...
6016    {930, 3685, 1639, 3245, 339, 3894, 3129, 3834,...
6028                                               {3000}
6040    {1921, 1673, 2571, 2575, 2068, 1947, 3362, 272...
Name: movie_id, Length: 1762, dtype: object

user_result_dict에 최근 본 선호 영화 목록 추가

In [70]:
has_rec_user_cnt = 0

for user, liked_movies in user_test_like_list.iteritems():
  if user in user_result_dict.keys():
    user_result_dict[user]['liked_movies'] = liked_movies
    has_rec_user_cnt += 1

print(has_rec_user_cnt)

1762


<ipython-input-70-baef8eed1d41>:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for user, liked_movies in user_test_like_list.iteritems():


In [71]:
user_result_dict = pd.DataFrame(user_result_dict).T.rename({'recent_watch_association_recommendations': 'recent_watch'}, axis=1).T.to_dict()

## Top K MAP, Recall 계산하기

[계산할 대상]
- 'age'
- 'high_score'
- 'genre'
- 'year'
- 'high_view'
- 'total_recommendations'

[비교 대상]
- 'liked_movies'

In [86]:
test_user_id = 1
column = 'high_score'
top_k = 30

In [185]:
def valid_recommend(test_user_id, column, k):
  predict_topk_list = list(user_result_dict[test_user_id][column])[:k] # 예측
  prefer_test = user_result_dict[test_user_id]['liked_movies'] # 실제

  if type(prefer_test) == float:
    return 0, 0

  prefer_test_list = list(prefer_test)
  true_positive = [pv for pv in predict_topk_list if pv in prefer_test_list]
  precision = len(true_positive) / len(predict_topk_list)
  recall = len(true_positive) / len(prefer_test_list)

  return precision, recall


In [186]:
import matplotlib.pyplot as plt

def draw_valid_graph(test_user_id, column, top_k, show='both'):
  p_ls, r_ls = [], []
  for k in range(1, top_k):
    p, r = valid_recommend(test_user_id, column, k)
    p_ls.append(p)
    r_ls.append(r)
  if show == 'precision':
    plt.plot(range(1, top_k), p_ls)
  elif show == 'recall':
    plt.plot(range(1, top_k), r_ls)
  else:
    plt.plot(range(1, top_k), p_ls)
    plt.plot(range(1, top_k), r_ls)

In [187]:
import numpy as np

In [199]:
def find_best_recommend_result(test_user_id, top_k_min=10, top_k_max=30):
  cols = ['age', 'high_score', 'genre', 'year', 'high_view', 'total_recommendations']

  cols_best_result = {}
  for c in cols:
    precision_ls = [valid_recommend(test_user_id, c, i)[0] for i in range(top_k_min,top_k_max)]
    recall_ls = [valid_recommend(test_user_id, c, i)[1] for i in range(top_k_min,top_k_max)]

    cols_best_result[c] = {}
    cols_best_result[c]['best_precision_k'] = top_k_min+np.argmax(precision_ls)
    cols_best_result[c]['best_precision_result'] = precision_ls[np.argmax(precision_ls)] # k개 추천했을때, 가장 높은 정밀도
    cols_best_result[c]['best_recall_k'] = top_k_min+np.argmax(recall_ls)
    cols_best_result[c]['best_recall_result'] = precision_ls[np.argmax(recall_ls)] # k개 추천했을때, 가장 높은 재현율

  return cols_best_result

### 전체 성능 평가

컬럼 별 전체 성능 평가 평균 보기

In [212]:
def get_mean_result_by_column(column, eval='both'):
  precision_result, recall_result = 0, 0
  nan_cnt = 0

  for user_id in user_result_dict.keys():
    user_df = pd.DataFrame(find_best_recommend_result(user_id)).T
    pr = user_df.loc[column]['best_precision_result']
    rr = user_df.loc[column]['best_recall_result']

    if pr == 0 and rr == 0:
      nan_cnt += 1

    else:
      precision_result += pr
      recall_result += rr

  eval_user_cnt = len(user_result_dict.keys())-nan_cnt

  if eval == 'precision':
    if eval_user_cnt == 0:
      return 0
    else:
      return precision_result / eval_user_cnt

  elif eval == 'recall':
    if eval_user_cnt == 0:
      return 0
    else:
      return recall_result / eval_user_cnt

  else:
    if eval_user_cnt == 0:
      return 0, 0
    else:
      return precision_result / eval_user_cnt, recall_result / eval_user_cnt

In [213]:
cols = ['age', 'high_score', 'genre', 'year', 'high_view', 'total_recommendations']

In [ ]:
for c in ['genre', 'year']:
  # try:
  p, r = get_mean_result_by_column(c)
  print(c, '평균')
  print('-'*30)
  print('정밀도:', p)
  print('재현율:', r)
  print()

  # except Exception as e:
  #   print(c, '평가 중 오류 발생')